
# Hypo/Hyperglycemia detection from ECG (paper replication)

Questa notebook replica in modo pratico il modello "Hypoglycemia and hyperglycemia detection using ECG: A multi-threshold based personalized fusion model" applicato al dataset **d1namo-ecg-glucose-data**. La struttura segue il paper:

- Obiettivo: rilevare episodi di **ipoglicemia (<70 mg/dL)** e **iperglicemia (>180 mg/dL)** a partire dall'ECG.
- Idea: combinare **morphology (intra-beat)** e **HRV (inter-beat)** con modelli personalizzati per paziente.
- Due task separati (hypo / hyper) e modello di **fusion multi-soglia**.

Ogni sezione ha celle Markdown + codice breve, con stampe e plot di controllo. I path e la logica di lettura riprendono `lab-1.ipynb`.


## 1) Setup & Config


Configurazione generale con import principali, seed fisso e parametri paper-like:
- path dataset coerenti con `lab-1.ipynb`
- frequenza ECG 250 Hz, bandpass 3-45 Hz
- soglia qualità HRConfidence (HRC) > 90
- finestre da 1 minuto per HRV e 5-fold CV temporale


In [ ]:

# Install neurokit2 (per R-peaks, HRV)
import sys, subprocess, importlib
try:
    import neurokit2 as nk
except ImportError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'neurokit2'])
    import neurokit2 as nk

import os, glob, re, math, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import defaultdict
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_fscore_support, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

DATA_ROOT = "/kaggle/input/d1namo-ecg-glucose-data"
DIAB_ECG_ROOT = os.path.join(DATA_ROOT, "diabetes_subset_ecg_data", "diabetes_subset_ecg_data")
HEAL_ECG_ROOT = os.path.join(DATA_ROOT, "healthy_subset_ecg_data", "healthy_subset_ecg_data")
DIAB_GLUCOSE_ROOT = os.path.join(DATA_ROOT, "diabetes_subset_pictures-glucose-food-insulin",
                                 "diabetes_subset_pictures-glucose-food-insulin")

# Parametri chiave (paper-like)
FS = 250  # Hz
BANDPASS = (3.0, 45.0)
HRC_THRESHOLD = 90  # percent
WIN_BEAT_SEC = 60  # HRV windows 1 min non-overlapping
N_FOLDS = 5
HYPO_THRESHOLDS = [55, 60, 65, 70, 75, 80, 85, 90]
HYPER_THRESHOLDS = [150, 165, 180, 200, 225, 250]

print("Imports OK | Seed=", SEED)
print("DATA_ROOT:", DATA_ROOT)
print("FS=", FS, "| BANDPASS=", BANDPASS, "| HRC cut=", HRC_THRESHOLD)


## 2) Data loading (riprende lab-1)


Caricamento dei file Kaggle D1NAMO:
- elenco ECG diabetici/sani e glucose.csv
- helper per estrarre ID paziente e sessione (stile `lab-1`)
- bandpass identico al notebook base
- scelta automatica del file glucose con massima overlap temporale rispetto alla sessione ECG
Stampe: numero file, esempi di path, pazienti e sessioni disponibili.


In [ ]:

def list_files(root, pattern="*.csv", max_show=5):
    files = sorted(glob.glob(os.path.join(root, "**", pattern), recursive=True))
    print(f"Root: {root} | found {len(files)} files")
    for f in files[:max_show]:
        print(" -", f)
    return files

diab_ecg_files = list_files(DIAB_ECG_ROOT, pattern="*_ECG.csv", max_show=3)
heal_ecg_files = list_files(HEAL_ECG_ROOT, pattern="*_ECG.csv", max_show=3)

glucose_files = sorted(glob.glob(os.path.join(DIAB_GLUCOSE_ROOT, "**", "glucose.csv"), recursive=True))
print("Glucose files:", len(glucose_files))
for f in glucose_files[:3]:
    print(" -", f)

assert len(diab_ecg_files) > 0, "No ECG files found (diabetes subset)."
assert len(glucose_files) > 0, "No glucose.csv files found."


In [ ]:

# Helpers to parse subject/session (from lab-1)
def parse_subject_id(path):
    parts = path.replace("\", "/").split("/")
    for p in parts:
        if re.fullmatch(r"\d{3}", p):
            return p
    return None

def parse_session_id(path):
    parts = path.replace("\", "/").split("/")
    for p in parts:
        if re.fullmatch(r"\d{4}_\d{2}_\d{2}-\d{2}_\d{2}_\d{2}", p):
            return p
    return None

def parse_session_datetime(sess_str):
    date_part, time_part = sess_str.split("-")
    hh, mm, ss = time_part.split("_")
    y, a, b = date_part.split("_")
    dt_A = datetime(int(y), int(a), int(b), int(hh), int(mm), int(ss))  # YYYY_MM_DD
    dt_B = datetime(int(y), int(b), int(a), int(hh), int(mm), int(ss))  # YYYY_DD_MM
    return dt_A, dt_B

print("Example subject/session:")
print(parse_subject_id(diab_ecg_files[0]), parse_session_id(diab_ecg_files[0]))


In [ ]:

from scipy.signal import butter, filtfilt

def butter_bandpass(lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype="band")
    return b, a

def apply_bandpass(x, fs=FS, lowcut=BANDPASS[0], highcut=BANDPASS[1], order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, x)


def load_ecg_file(path, fs=FS):
    df = pd.read_csv(path)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if "EcgWaveform" in df.columns:
        sig = pd.to_numeric(df["EcgWaveform"], errors="coerce").values
    elif len(num_cols) > 0:
        sig = pd.to_numeric(df[num_cols[0]], errors="coerce").values
    else:
        raise ValueError("No numeric ECG column found.")

    raw_clean = sig[~np.isnan(sig)]
    if len(raw_clean) > 0 and (np.nanpercentile(raw_clean, 95) > 50):
        ecg_mv = (sig.astype(float) - 1024.0) / 200.0
        conv_used = True
    else:
        ecg_mv = sig.astype(float)
        conv_used = False
    ecg_filt = apply_bandpass(np.nan_to_num(ecg_mv), fs=fs)
    return pd.DataFrame({"ecg_mv": ecg_mv, "ecg_filt": ecg_filt}), conv_used


def load_glucose_file(glc_path, dayfirst=False):
    df = pd.read_csv(glc_path)
    if ("date" in df.columns) and ("time" in df.columns):
        df["Time"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str),
                                     errors="coerce", dayfirst=dayfirst)
    elif "Time" in df.columns:
        df["Time"] = pd.to_datetime(df["Time"], errors="coerce", dayfirst=dayfirst)
    else:
        tc = None
        for c in df.columns:
            if "time" in c.lower() or "date" in c.lower():
                tc = c; break
        if tc is None:
            return None
        df["Time"] = pd.to_datetime(df[tc], errors="coerce", dayfirst=dayfirst)

    if "glucose" in df.columns:
        df["glucose_mg_dL"] = pd.to_numeric(df["glucose"], errors="coerce") * 18.0
    else:
        gnum = df.select_dtypes(include=[np.number]).columns.tolist()
        if len(gnum) == 0:
            return None
        df["glucose_mg_dL"] = pd.to_numeric(df[gnum[0]], errors="coerce")

    df = df[["Time", "glucose_mg_dL"]].dropna().sort_values("Time").reset_index(drop=True)
    return df if len(df) else None


def choose_glucose_for_ecg(ecg_path, glucose_files):
    sess = parse_session_id(ecg_path)
    dtA, dtB = parse_session_datetime(sess)
    df_ecg, _ = load_ecg_file(ecg_path)
    dur = timedelta(seconds=(len(df_ecg)-1)/FS)
    best = None
    for p in glucose_files:
        for dayfirst in [False, True]:
            dfg = load_glucose_file(p, dayfirst=dayfirst)
            if dfg is None:
                continue
            g0, g1 = dfg["Time"].min(), dfg["Time"].max()
            def ov(t0,t1,u0,u1):
                ov0, ov1 = max(t0,u0), min(t1,u1)
                return max(0.0,(ov1-ov0).total_seconds())/3600.0
            ovA = ov(dtA, dtA+dur, g0, g1)
            ovB = ov(dtB, dtB+dur, g0, g1)
            ovh = max(ovA, ovB)
            which = "A" if ovA>=ovB else "B"
            cand = (ovh, p, dayfirst, which, len(dfg))
            if (best is None) or (cand[0] > best[0]):
                best = cand
    assert best is not None, "No glucose candidate matched."
    ovh, p, dayfirst, which, n = best
    print(f"Chosen glucose {p} | overlap_h={ovh:.2f} | dayfirst={dayfirst} | interp={which} | n={n}")
    return load_glucose_file(p, dayfirst=dayfirst), (which, dtA, dtB)

print("Bandpass + loaders ready.")

In [ ]:

pat_sessions = defaultdict(list)
for p in diab_ecg_files:
    subj = parse_subject_id(p)
    sess = parse_session_id(p)
    pat_sessions[subj].append(p)

print("Patients:", len(pat_sessions))
for k,v in list(pat_sessions.items())[:5]:
    print(k, "sessions", len(v))


## 3) Preprocessing ECG (paper-like)


Pipeline paper-like:
- R-peak detection con `nk.ecg_process`
- Qualità battito via `nk.ecg_quality` (HRC) e filtro > 90
- Delineazione fiduciali P/Q/S/T con wavelet; scarto se manca almeno un punto
- Plot di esempio 10s con R-peaks e percentuale di beat tenuti/scartati


In [ ]:

import neurokit2 as nk


def detect_peaks_and_quality(ecg_df, fs=FS, hrc_cut=HRC_THRESHOLD):
    signals, info = nk.ecg_process(ecg_df["ecg_filt"], sampling_rate=fs)
    rpeaks = info["ECG_R_Peaks"]
    quality = nk.ecg_quality(ecg_df["ecg_filt"], rpeaks=rpeaks, sampling_rate=fs, method="zhao2018")
    quality_pct = np.clip(quality * 100.0, 0, 100)
    good = np.where(quality_pct > hrc_cut)[0]
    delinea, waves = nk.ecg_delineate(ecg_df["ecg_filt"], rpeaks=rpeaks, sampling_rate=fs, method="dwt")
    fiducials = {
        "rpeaks": rpeaks,
        "p_onsets": waves.get("ECG_P_Onsets", []),
        "p_peaks": waves.get("ECG_P_Peaks", []),
        "q_peaks": waves.get("ECG_Q_Peaks", []),
        "s_peaks": waves.get("ECG_S_Peaks", []),
        "t_peaks": waves.get("ECG_T_Peaks", []),
    }
    return quality_pct, good, fiducials

_ecg_raw, used_conv = load_ecg_file(diab_ecg_files[0])
qual, good_idx, fid = detect_peaks_and_quality(_ecg_raw)
print("Beats detected:", len(fid["rpeaks"]), "| good beats %:", 100*len(good_idx)/max(1,len(fid["rpeaks"])) )

win_sec = 10
i0 = 0
i1 = min(len(_ecg_raw), FS*win_sec)
plt.figure(figsize=(14,3))
plt.plot(_ecg_raw.index/FS, _ecg_raw["ecg_filt"], label="ECG filt")
peaks_in = [p for p in fid["rpeaks"] if (p>=i0 and p<i1)]
plt.scatter(np.array(peaks_in)/FS, _ecg_raw["ecg_filt"].iloc[peaks_in], color='r', s=12, label='R')
plt.xlim(i0/FS, i1/FS)
plt.title("Raw ECG with R-peaks (demo)")
plt.xlabel("seconds")
plt.legend();
plt.show()


## 4) Feature extraction (morphology + HRV + labeling)


### 4.1 Morphology beat-level
- intervalli PQ/QS/ST/PR/RT (sec)
- ampiezze P/Q/R/S/T e slope tra picchi
- RR e HR aggiunti per beat; timestamp assoluto

### 4.2 HRV interval-level (1 min)
- 18 feature time-domain da NeuroKit2 su finestre 1-min non-overlapping

### 4.3 Labeling
- associa ogni beat/finestra al valore CGM successivo (forward nearest)
- task separati: hypoglycemia (<70), hyperglycemia (>180)

### 4.4 Interval features da beat probabilities
- percentuale beat positivi, longest positive sequence
- media probabilità + distribuzioni per bin (0,0.2], ... (0.8,1]
- hour-of-day encoding ciclico


In [ ]:

from itertools import zip_longest


def build_beat_level_features(ecg_df, fid, quality_pct, fs=FS, hrc_cut=HRC_THRESHOLD):
    beats = []
    rpeaks = fid["rpeaks"]
    def nearest(arr, idx):
        return arr[np.argmin(np.abs(np.array(arr) - idx))] if len(arr) else None

    for idx in rpeaks:
        q = nearest(fid["q_peaks"], idx)
        s = nearest(fid["s_peaks"], idx)
        p = nearest(fid["p_peaks"], idx)
        t = nearest(fid["t_peaks"], idx)
        hrc = quality_pct[idx] if idx < len(quality_pct) else np.nan
        if (pd.isna(hrc)) or (hrc < hrc_cut):
            reason = "low_hrc"
        elif any(v is None for v in [p,q,s,t]):
            reason = "missing_fiducial"
        else:
            reason = None
        if reason:
            beats.append({"r_idx": idx, "reason": reason, "keep": False})
            continue
        pq = (idx - p)/fs
        qs = (s - q)/fs if s and q else np.nan
        st = (t - s)/fs if t and s else np.nan
        pr = (idx - p)/fs if p else np.nan
        rt = (t - idx)/fs if t else np.nan
        amp_r = ecg_df["ecg_filt"].iloc[idx]
        amp_p = ecg_df["ecg_filt"].iloc[p]
        amp_q = ecg_df["ecg_filt"].iloc[q]
        amp_s = ecg_df["ecg_filt"].iloc[s]
        amp_t = ecg_df["ecg_filt"].iloc[t]
        slope_qr = (amp_r - amp_q)/((idx-q)/fs)
        slope_rs = (amp_s - amp_r)/((s-idx)/fs)
        slope_st = (amp_t - amp_s)/((t-s)/fs)
        beats.append({
            "r_idx": idx,
            "keep": True,
            "reason": "ok",
            "time": idx/fs,
            "pq_int": pq,
            "qs_int": qs,
            "st_int": st,
            "pr_int": pr,
            "rt_int": rt,
            "amp_r": amp_r,
            "amp_p": amp_p,
            "amp_q": amp_q,
            "amp_s": amp_s,
            "amp_t": amp_t,
            "slope_qr": slope_qr,
            "slope_rs": slope_rs,
            "slope_st": slope_st,
        })
    df_beats = pd.DataFrame(beats)
    df_beats = df_beats[df_beats["keep"]].reset_index(drop=True)
    df_beats["rr"] = df_beats["r_idx"].diff()/fs
    df_beats["hr"] = 60.0/df_beats["rr"]
    df_beats["timestamp"] = pd.to_datetime(df_beats["time"], unit="s")
    print("Beat features shape", df_beats.shape, "| kept %", 100*len(df_beats)/max(1,len(fid["rpeaks"])) )
    return df_beats

beat_features_demo = build_beat_level_features(_ecg_raw, fid, qual)
beat_features_demo.head()


In [ ]:


def compute_hrv_windows(fid, fs=FS, win_sec=WIN_BEAT_SEC):
    rpeaks = np.array(fid["rpeaks"], dtype=int)
    if len(rpeaks) < 4:
        return pd.DataFrame()
    times = rpeaks / fs
    starts = np.arange(0, times.max()+1, win_sec)
    rows = []
    for s in starts:
        e = s + win_sec
        mask = (times>=s) & (times<e)
        seg = rpeaks[mask]
        if len(seg) < 4:
            continue
        hrv = nk.hrv_time({"RPeaks": seg}, sampling_rate=fs, show=False)
        hrv["t_start"] = s
        hrv["t_end"] = e
        rows.append(hrv)
    if not rows:
        return pd.DataFrame()
    df_hrv = pd.concat(rows, ignore_index=True)
    df_hrv["window_start"] = pd.to_datetime(df_hrv["t_start"], unit="s")
    print("HRV windows", df_hrv.shape)
    return df_hrv

hrv_demo = compute_hrv_windows(fid)
hrv_demo.head()


In [ ]:


def label_by_glucose(times, glucose_df, task="hypo"):
    glc_times = glucose_df["Time"].values.astype('datetime64[ns]')
    glc_vals = glucose_df["glucose_mg_dL"].values
    labels = []
    for t in times:
        idx = glc_times.searchsorted(t)
        if idx >= len(glc_times):
            labels.append(np.nan)
            continue
        g = glc_vals[idx]
        if task == "hypo":
            labels.append(1 if g < 70 else 0)
        else:
            labels.append(1 if g > 180 else 0)
    return np.array(labels)

# apply to beat and interval levels
glc_demo, (which, dtA, dtB) = choose_glucose_for_ecg(diab_ecg_files[0], glucose_files)

beat_features_demo["label_hypo"] = label_by_glucose(beat_features_demo["timestamp"].values, glc_demo, task="hypo")
beat_features_demo["label_hyper"] = label_by_glucose(beat_features_demo["timestamp"].values, glc_demo, task="hyper")

if not hrv_demo.empty:
    hrv_demo["label_hypo"] = label_by_glucose(hrv_demo["window_start"].values, glc_demo, task="hypo")
    hrv_demo["label_hyper"] = label_by_glucose(hrv_demo["window_start"].values, glc_demo, task="hyper")

print("Class balance hypo beats:")
print(beat_features_demo["label_hypo"].value_counts(dropna=False))


In [ ]:


def interval_features_from_probs(df_beats, prob_col="prob", win_sec=WIN_BEAT_SEC):
    if prob_col not in df_beats:
        raise ValueError("prob_col missing")
    df = df_beats.copy()
    df["window_start"] = (df["timestamp"].view('int64') // (win_sec*1e9)).astype(int)
    rows = []
    for w, g in df.groupby("window_start"):
        probs = g[prob_col].values
        thr = (probs>0.5).astype(int)
        longest = 0
        current = 0
        for v in thr:
            if v==1:
                current += 1
                longest = max(longest, current)
            else:
                current = 0
        bins = pd.cut(probs, bins=[0,0.2,0.4,0.6,0.8,1.0], right=True, include_lowest=True)
        bin_counts = bins.value_counts(normalize=True).sort_index()
        row = {
            "window_start": pd.to_datetime(w*win_sec, unit="s"),
            "prob_mean": np.nanmean(probs),
            "beat_pos_frac": thr.mean(),
            "longest_pos_seq": longest,
        }
        for b, val in bin_counts.items():
            row[f"bin_{b.left:.1f}_{b.right:.1f}"] = val
        hod = row["window_start"].hour + row["window_start"].minute/60
        row["hod_sin"] = math.sin(2*math.pi*hod/24)
        row["hod_cos"] = math.cos(2*math.pi*hod/24)
        rows.append(row)
    return pd.DataFrame(rows).sort_values("window_start")

beat_features_demo["prob_demo"] = np.clip(np.random.beta(2,5, size=len(beat_features_demo)), 0, 1)
interval_from_probs_demo = interval_features_from_probs(beat_features_demo, prob_col="prob_demo")
interval_from_probs_demo.head()
print("Interval prob features:", interval_from_probs_demo.shape)

## 5) Temporal cross-validation (1h blocks, 5-fold)


Suddivisione temporale:
- ordina per tempo e crea blocchi da 1h
- shuffle dei blocchi e assegnazione a 5 fold
- obiettivo: ogni fold include positivi/negativi quando disponibili
- stampa numero di esempi per fold


In [ ]:


def build_temporal_folds(df, time_col, label_col, n_folds=N_FOLDS, block_minutes=60):
    df = df.dropna(subset=[label_col]).copy()
    df = df.sort_values(time_col).reset_index(drop=True)
    df["block"] = (df[time_col].view('int64') // (block_minutes*60*1e9)).astype(int)
    blocks = df["block"].unique()
    rng = np.random.default_rng(SEED)
    rng.shuffle(blocks)
    folds = {}
    split = np.array_split(blocks, n_folds)
    for i, blk in enumerate(split):
        folds[i] = df[df["block"].isin(blk)].index.tolist()
    print({k: len(v) for k,v in folds.items()})
    return folds

folds_demo = build_temporal_folds(beat_features_demo, "timestamp", "label_hypo")


## 6) Modelli (MBeat, intervalli, fusion)


Modelli paper-like per paziente e task:
- **MBeat**: RF su feature morfologiche beat-level
- **MMV/MMorph**: majority su beat o RF su feature di finestra da MBeat
- **MHRV**: RF su feature HRV 1-min
- **MMorph+HRV**: concatenazione feature di finestra + HRV
- **Fusion multi-soglia (MFMorph+HRV)**: train multipli MBeat a soglie diverse, estrai feature di Tab.2 e concatena con HRV
Funzioni riusabili per training su fold temporali e output probabilistici.


In [ ]:


def train_beat_model(df_beats, label_col, folds):
    feats = [c for c in df_beats.columns if c not in [label_col, "timestamp", "time", "r_idx", "keep", "reason"] and not c.startswith("prob_")]
    models = {}
    fold_preds = []
    for k, idx_val in folds.items():
        train_idx = df_beats.index.difference(idx_val)
        val_idx = idx_val
        clf = RandomForestClassifier(n_estimators=200, random_state=SEED, class_weight="balanced")
        clf.fit(df_beats.loc[train_idx, feats], df_beats.loc[train_idx, label_col])
        proba = clf.predict_proba(df_beats.loc[val_idx, feats])[:,1]
        fold_preds.append(pd.DataFrame({"idx": val_idx, "proba": proba}))
        models[k] = clf
    preds = pd.concat(fold_preds).set_index("idx").sort_index()["proba"].values
    out = df_beats.copy()
    out[f"prob_{label_col}"] = preds
    return models, out


def train_interval_rf(df_feats, label_col, folds):
    feats = [c for c in df_feats.columns if c not in [label_col, "window_start"]]
    models = {}
    preds = []
    for k, idx_val in folds.items():
        train_idx = df_feats.index.difference(idx_val)
        val_idx = idx_val
        clf = RandomForestClassifier(n_estimators=300, random_state=SEED, class_weight="balanced")
        clf.fit(df_feats.loc[train_idx, feats], df_feats.loc[train_idx, label_col])
        proba = clf.predict_proba(df_feats.loc[val_idx, feats])[:,1]
        preds.append(pd.DataFrame({"idx": val_idx, "proba": proba}))
        models[k] = clf
    pred_series = pd.concat(preds).set_index("idx").sort_index()["proba"].values
    df_out = df_feats.copy()
    df_out[f"prob_{label_col}"] = pred_series
    return models, df_out


def fusion_multi_threshold(df_beats, fid, glucose_df, task="hypo"):
    thresholds = HYPO_THRESHOLDS if task=="hypo" else HYPER_THRESHOLDS
    label_col = f"label_{task}"
    folds = build_temporal_folds(df_beats, "timestamp", label_col)
    interval_feat_list = []
    for thr in thresholds:
        df_tmp = df_beats.copy()
        df_tmp[label_col] = label_by_glucose(df_tmp["timestamp"].values, glucose_df, task=task)
        models, df_pred = train_beat_model(df_tmp, label_col, folds)
        df_pred[f"prob_thr_{thr}"] = df_pred[f"prob_{label_col}"]
        inter = interval_features_from_probs(df_pred.rename(columns={f"prob_{label_col}":"prob"}), prob_col="prob")
        inter[label_col] = label_by_glucose(inter["window_start"].values, glucose_df, task=task)
        inter = inter.add_prefix(f"thr{thr}_")
        inter = inter.rename(columns={f"thr{thr}_window_start":"window_start", f"thr{thr}_{label_col}": label_col})
        interval_feat_list.append(inter)
    fused = interval_feat_list[0][["window_start", label_col]].copy()
    for inter in interval_feat_list:
        fused = fused.merge(inter.drop(columns=[label_col]), on="window_start", how="left")
    hrv = compute_hrv_windows(fid)
    if not hrv.empty:
        fused = fused.merge(hrv.drop(columns=["label_hypo","label_hyper"] if "label_hypo" in hrv.columns else []), on="window_start", how="left")
    folds_int = build_temporal_folds(fused, "window_start", label_col)
    models_fused, fused_pred = train_interval_rf(fused.fillna(0), label_col, folds_int)
    return fused_pred
print("Interval prob features:", interval_from_probs_demo.shape)
print("Model helpers ready.")

## 7) Metriche & risultati


Metriche calcolate:
- AUC ROC principale + sensitivity, specificity, precision, F1
- Plot ROC di esempio; estendibile a barre media/std tra pazienti e confronti fusion/baseline
- Esecuzione demo su un paziente per mostrare pipeline end-to-end


In [ ]:


def compute_metrics(df, label_col, prob_col):
    y = df[label_col].values
    p = df[prob_col].values
    auc = roc_auc_score(y, p) if len(np.unique(y))>1 else np.nan
    preds = (p>=0.5).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, preds).ravel()
    prec, rec, f1, _ = precision_recall_fscore_support(y, preds, average='binary', zero_division=0)
    spec = tn / (tn + fp) if (tn+fp)>0 else np.nan
    return {"AUC": auc, "sens": rec, "spec": spec, "prec": prec, "f1": f1}

patient = list(pat_sessions.keys())[0]
first_ecg = pat_sessions[patient][0]
print("Patient", patient, "file", first_ecg)

df_ecg, _ = load_ecg_file(first_ecg)
qual, good_idx, fid = detect_peaks_and_quality(df_ecg)
beats = build_beat_level_features(df_ecg, fid, qual)

glc_df, (which, dtA, dtB) = choose_glucose_for_ecg(first_ecg, glucose_files)
sess_dt = dtA if which=="A" else dtB
beats["timestamp"] = pd.to_datetime([sess_dt + timedelta(seconds=t) for t in beats["time"]])
beats["label_hypo"] = label_by_glucose(beats["timestamp"].values, glc_df, task="hypo")

folds = build_temporal_folds(beats, "timestamp", "label_hypo")
beat_models, beat_pred = train_beat_model(beats, "label_hypo", folds)

interval_probs = interval_features_from_probs(beat_pred.rename(columns={"prob_label_hypo":"prob"}), prob_col="prob")
interval_probs["label_hypo"] = label_by_glucose(interval_probs["window_start"].values, glc_df, task="hypo")
folds_int = build_temporal_folds(interval_probs, "window_start", "label_hypo")
interval_models, interval_pred = train_interval_rf(interval_probs.fillna(0), "label_hypo", folds_int)

print("Beat-level metrics:", compute_metrics(beat_pred.dropna(subset=["label_hypo"]), "label_hypo", "prob_label_hypo"))
print("Interval-level metrics (MMorph):", compute_metrics(interval_pred.dropna(subset=["label_hypo"]), "label_hypo", "prob_label_hypo"))

fpr, tpr, _ = roc_curve(interval_pred["label_hypo"], interval_pred["prob_label_hypo"])
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='MMorph hypo')
plt.plot([0,1],[0,1],'--', color='gray')
plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC interval model')
plt.legend(); plt.show()
print("Interval prob features:", interval_from_probs_demo.shape)

## 8) Interpretabilità (feature importance)


Importanza feature:
- importance da RandomForest (impurity)
- fallback SHAP (se installato) per summary plot; in caso di errore viene notificato


In [ ]:


def plot_feature_importance(model, feature_names, topk=15, title="RF importance"):
    imp = model.feature_importances_
    order = np.argsort(imp)[::-1][:topk]
    plt.figure(figsize=(6,4))
    plt.barh(range(len(order)), imp[order][::-1])
    plt.yticks(range(len(order)), [feature_names[i] for i in order][::-1])
    plt.title(title)
    plt.tight_layout()
    plt.show()

if 'interval_models' in globals() and len(interval_models)>0:
    any_model = list(interval_models.values())[0]
    feats = [c for c in interval_pred.columns if c not in ["label_hypo", "window_start", "prob_label_hypo"]]
    plot_feature_importance(any_model, feats, title="Interval RF - feature importance")

try:
    import shap
    explainer = shap.TreeExplainer(any_model)
    shap_values = explainer.shap_values(interval_pred[feats].iloc[:200])
    shap.summary_plot(shap_values[1], interval_pred[feats].iloc[:200], max_display=15)
except Exception as e:
    print("SHAP skipped (", e, ")")
